## Phần 1: Giới thiệu và Cài đặt

In [1]:
pip install -U spacy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Phần 2: Phân tích câu và Trực quan hóa

### 2.1. Tải mô hình và phân tích câu

In [2]:
import spacy
from spacy import displacy

In [3]:
# Tải mô hình tiếng Anh đã cài đặt
# Sử dụng en_core_web_md vì nó chứa các vector từ và cây cú pháp đầy đủ
nlp = spacy.load("en_core_web_md")
# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."
# Phân tích câu với pipeline của spaCy
doc = nlp(text)

### 2.2. Trực quan hóa cây phụ thuộc

In [4]:
# Tùy chọn để hiển thị trong trình duyệt
options = {"compact": True, "color": "blue", "font": "Source Sans Pro"}
# Khởi chạy server tại http://127.0.0.1:5000
# Bạn có thể truy cập địa chỉ này trên trình duyệt để xem cây phụ thuộc
# Nhấn Ctrl+C trong terminal để dừng server
displacy.serve(doc, style="dep")

c:\Users\ADM\anaconda3\Lib\site-packages\spacy\displacy\__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


#### Trả lời câu hỏi:
**1. Từ nào là gốc (ROOT) của câu?**

Từ **jumps** là gốc (ROOT) của câu "**The quick brown fox jumps over the lazy dog.**"

**2. `jumps` có những từ phụ thuộc (dependent) nào? Các quan hệ đó là gì?**

| Từ phụ thuộc | Quan hệ cú pháp       | Mô tả                              |
|--------------|------------------------|------------------------------------|
| fox          | nsubj (nominal subject) | Chủ ngữ danh từ của động từ        |
| over         | prep (prepositional modifier) | Cụm giới từ bổ nghĩa cho động từ   |

**3. `fox` là head của những từ nào?**

| Từ phụ thuộc | Quan hệ cú pháp           | Mô tả                          |
|--------------|----------------------------|--------------------------------|
| The          | det (determiner)           | Mạo từ xác định                |
| quick        | amod (adjectival modifier) | Tính từ bổ nghĩa danh từ       |
| brown        | amod (adjectival modifier) | Tính từ bổ nghĩa danh từ       |

## Phần 3: Truy cập các thành phần trong cây phụ thuộc

In [5]:
# Lấy một câu khác để phân tích
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)

In [6]:
# In ra thông tin của từng token
print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)

for token in doc:
    # Trích xuất các thuộc tính
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


## Phần 4: Duyệt cây phụ thuộc để trích xuất thông tin

### 4.1. Bài toán: Tìm chủ ngữ và tân ngữ của một động từ

In [7]:
text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)

In [8]:
for token in doc:
    # Chỉ tìm các động từ
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""
        
        # Tìm chủ ngữ (nsubj) và tân ngữ (dobj) trong các con của động từ
        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text
        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")

Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


### 4.2. Bài toán: Tìm các tính từ bổ nghĩa cho một danh từ

In [9]:
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)

In [10]:
for token in doc:
    # Chỉ tìm các danh từ
    if token.pos_ == "NOUN":
        adjectives = []
        # Tìm các tính từ bổ nghĩa (amod) trong các con của danh từ
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)
                
        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi các tính từ: {adjectives}")

Danh từ 'cat' được bổ nghĩa bởi các tính từ: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi các tính từ: ['warm']


## Phần 5: Bài tập tự luyện

### Bài 1: Tìm động từ chính của câu

In [11]:
def find_main_verb(doc):
    for token in doc:
        if token.dep_ == "ROOT" and token.pos_ in ("VERB", "AUX"):
            return token

    for token in doc:
        if token.dep_ == "ROOT":
            return token
    return None

In [12]:
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)
main_verb = find_main_verb(doc)
print("Câu:", text)
print("Động từ chính:", main_verb, "- POS:", main_verb.pos_, "- DEP:", main_verb.dep_)

Câu: The quick brown fox jumps over the lazy dog.
Động từ chính: jumps - POS: VERB - DEP: ROOT


### Bài 2: Trích xuất các cụm danh từ (Noun Chunks)

In [13]:
MODIFIER_DEPS = {"det", "amod", "compound", "poss", "nummod"}

def build_noun_chunk(token):
    left_index = token.i
    right_index = token.i

    for left in token.lefts:
        if left.dep_ in MODIFIER_DEPS:
            left_index = min(left_index, left.i)

    for right in token.rights:
        if right.dep_ in MODIFIER_DEPS:
            right_index = max(right_index, right.i)

    return token.doc[left_index : right_index + 1]


def custom_noun_chunks(doc):
    chunks = []
    for token in doc:
        if token.pos_ in ("NOUN", "PROPN", "PRON") and token.dep_ not in ("compound", "amod"):
            span = build_noun_chunk(token)
            chunks.append(span)
    return chunks

In [14]:
text = "The big, fluffy white cat is sleeping on the warm mat in the living room."
doc = nlp(text)
print("Câu:", text)
print("Các cụm danh từ (tự trích xuất):")
for span in custom_noun_chunks(doc):
    print("-", span.text)

print("Các cụm danh từ (spaCy noun_chunks):")
for chunk in doc.noun_chunks:
    print("-", chunk.text)

Câu: The big, fluffy white cat is sleeping on the warm mat in the living room.
Các cụm danh từ (tự trích xuất):
- The big, fluffy white cat
- the warm mat
- the living room
Các cụm danh từ (spaCy noun_chunks):
- The big, fluffy white cat
- the warm mat
- the living room


### Bài 3: Tìm đường đi ngắn nhất trong cây

In [15]:
def get_path_to_root(token):
    path = []
    current = token
    while True:
        path.append(current)
        if current.head == current:
            break
        current = current.head
    return path

In [16]:
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

for token in doc:
    if token.text == "lazy":
        target = token
        break

print("Câu:", text)
print("Token bắt đầu:", target.text)
print("Đường đi đến ROOT:")
path = get_path_to_root(target)
print(" -> ".join([t.text for t in path]))

Câu: The quick brown fox jumps over the lazy dog.
Token bắt đầu: lazy
Đường đi đến ROOT:
lazy -> dog -> over -> jumps
